In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [24]:
app = pd.read_csv('appRefine.csv')
pay = pd.read_csv('payRefine.csv')
PPDB = pd.read_csv('reFinePPDB.csv',dtype=str)

In [25]:
app.columns=['PANEL_ID', 'APP_CATEGORY', 'APP_USE_TME']

In [26]:
total = app.merge(pay, on= 'PANEL_ID').merge(PPDB, on= 'PANEL_ID')

In [27]:
total.loc[:,'APP_CATEGORY'] = total.loc[:,'APP_CATEGORY'].map(lambda x :'ETC' if pd.isna(x) else x)

In [28]:
total['CATEGORY_CODE'] =total['CATEGORY_CODE'].map(lambda x : str(int(x)))
total['family_cnt'] = total['family_cnt'].map(lambda x: int(x))

In [34]:
dummy_total = pd.get_dummies(total.drop(columns='PANEL_ID'))

In [35]:
#normalize
dummy_total['APP_USE_TME'] =dummy_total['APP_USE_TME'].map(lambda  x: (x-dummy_total['APP_USE_TME'].mean())/dummy_total['APP_USE_TME'].std())
dummy_total['PAY_CNT'] =dummy_total['PAY_CNT'].map(lambda  x: (x-dummy_total['PAY_CNT'].mean())/dummy_total['PAY_CNT'].std())
dummy_total['PAY_MEAN_AMOUNT'] =dummy_total['PAY_MEAN_AMOUNT'].map(lambda  x: (x-dummy_total['PAY_MEAN_AMOUNT'].mean())/dummy_total['PAY_MEAN_AMOUNT'].std())
dummy_total['family_cnt'] =dummy_total['family_cnt'].map(lambda  x: (x-dummy_total['family_cnt'].mean())/dummy_total['family_cnt'].std())

In [37]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(dummy_total)

In [38]:
dummy_total2 =dummy_total.copy()

In [39]:
dummy_total2['km_label'] = kmeans.labels_

In [42]:
dummy_total2.groupby('km_label').count()[['APP_USE_TME']]

,APP_USE_TME
km_label,
0,3195
1,246
2,1863
3,2357
4,6


In [43]:
total[dummy_total2['km_label']==4]

,PANEL_ID,APP_CATEGORY,APP_USE_TME,CATEGORY_CODE,Area_Name2,PAY_CNT,PAY_MEAN_AMOUNT,gender,age,loctown,is_marry,education,job,family_cnt,has_baby,has_student,has_silver,ann_income,home_cat,home_pay
1809,H0206932,SOCIAL,120930,401,서초구,430,36589426,1,4,1,2,6,6,4,0,0,0,5,2,2
2670,I0567936,ETC,255189,501,종로구,346,33987281,2,2,1,1,3,6,4,0,0,0,5,5,1
3089,I0850237,LIFESTYLE,203420,501,용산구,1098,26918838,2,5,1,1,1,6,3,0,0,1,3,5,1
3974,P0115421,SOCIAL,363810,1602,안산시상록구,80,18603065,2,6,1,2,4,997,3,0,0,1,3,2,1
7268,P0294857,SOCIAL,260052,101,군포시,372,55933893,2,3,1,2,4,6,4,1,0,0,4,2,2
7539,P0304340,SOCIAL,73550,401,양천구,13,20266831,1,4,1,2,4,6,3,0,0,0,5,2,1


In [47]:
dummy_total2.shape

(7667, 186)

In [48]:
total.shape

(7667, 20)

In [49]:
out = total.copy()
out['KNN_LABEL']=kmeans.labels_

In [51]:
out.to_csv('KNN_result.csv',index=0)